In [1]:
# ✅ 1. 필수 라이브러리 설치 (Colab에서만 필요)
!pip install --quiet gspread gspread_dataframe oauth2client google-api-python-client pandas

# ✅ 2. 라이브러리 불러오기 및 인증
import pandas as pd
import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials

# ✅ 3. Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')

# ✅ 4. 인증 파일 경로 (Colab 기준)
json_path = "/content/drive/My Drive/Key/credentials.json"

# ✅ gspread 인증
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
credentials = ServiceAccountCredentials.from_json_keyfile_name(json_path, scope)
gc = gspread.authorize(credentials)

# ✅ Google Drive API 인증
creds_googleapi = Credentials.from_service_account_file(json_path, scopes=["https://www.googleapis.com/auth/drive"])
drive_service = build('drive', 'v3', credentials=creds_googleapi)

# ✅ 병합 대상 Google Sheet IDs (지사별)
sheet_ids = {
    "중앙": "1Yn7Y_UFNKlfzHzHfH_e6FCR4M-GOqfGtoqXU-5gW5EE",
    "강북": "1ALiNS9LRI56I1OrF2KJTi4-N8Y6YRGxGBdKtfPRr30g",
    "서대문": "1nMbFmdYEB6zdViwA9iRwjLO9H0rsXfFs14um3LLzsfw",
    "고양": "18YhW6PUm6Yqgya31vWOjBuXSTQ054Kqs4uWbug1d99E",
    "남양주": "18S7r0M8XTLtZ8V-d08L7T60lK1BV8FuzYDGH6zox_pk",
    "의정부": "1kYu0qNoHgk7fXArZLCS-pCmZi-5XJSFZ0tWjkTA1I94",
    "강릉": "1WsgT5HoMOrDAPWXGh1e8wWiX2KzMo0BmekLHJ-h5_A8",
    "원주": "16rRZHK0S3pzG_95GxkAg0GZOmAm5okjz8nFIxmM4rpM"
}

# ✅ 병합할 시트명 (작은따옴표 정확히 사용)
target_sheet_name = "체납활동내역('25.04)"

# ✅ 병합 처리
merged_df = pd.DataFrame()

for branch, sheet_id in sheet_ids.items():
    try:
        sheet = gc.open_by_key(sheet_id)
        worksheet = sheet.worksheet(target_sheet_name)
        df = get_as_dataframe(worksheet, evaluate_formulas=True)
        df = df.dropna(how='all')
        df["지사명"] = branch
        merged_df = pd.concat([merged_df, df], ignore_index=True)
        print(f"✅ {branch} 병합 완료")
    except Exception as e:
        print(f"⚠️ {branch} 오류: {e}")

print("✅ 전체 병합 완료. 총 행 수:", len(merged_df))

# ✅ 체납 폴더 ID
folder_id = "1gIzD9Xd_agkSP16FfIj3oRuKZLydN2Of"

# ✅ Google Sheets 생성 및 저장
file_metadata = {
    'name': '병합결과_체납',
    'mimeType': 'application/vnd.google-apps.spreadsheet',
    'parents': [folder_id]
}
spreadsheet = drive_service.files().create(body=file_metadata, fields='id').execute()
spreadsheet_id = spreadsheet.get('id')
print(f"✅ 스프레드시트 생성 완료: https://docs.google.com/spreadsheets/d/{spreadsheet_id}")

# ✅ 데이터 기록
worksheet = gc.open_by_key(spreadsheet_id).sheet1
set_with_dataframe(worksheet, merged_df)
print("✅ 병합된 데이터 저장 완료!")


Mounted at /content/drive
✅ 중앙 병합 완료
✅ 강북 병합 완료
✅ 서대문 병합 완료
✅ 고양 병합 완료
✅ 남양주 병합 완료
✅ 의정부 병합 완료
✅ 강릉 병합 완료
✅ 원주 병합 완료
✅ 전체 병합 완료. 총 행 수: 2170
✅ 스프레드시트 생성 완료: https://docs.google.com/spreadsheets/d/1T1YJaI7FK2-q4wTnSHd8xd1L4Dmciw8fJynYf4M9K6Q
✅ 병합된 데이터 저장 완료!
